In [1]:
from pandas import read_csv

from scripts.cleaning_utils import drop_outliers, unite_floors_together
from scripts.analyse_utils import counter_columns

In [2]:
dataframe = read_csv("../data/raw/houses_madrid.csv")
dataframe.describe(include='all')

,Unnamed: 0,id,title,subtitle,sq_mt_built,sq_mt_useful,n_rooms,n_bathrooms,n_floors,sq_mt_allotment,...,energy_certificate,has_parking,has_private_parking,has_public_parking,is_parking_included_in_price,parking_price,is_orientation_north,is_orientation_west,is_orientation_south,is_orientation_east
count,21742.000000,21742.000000,21742,21742,21616.000000,8228.000000,21742.000000,21726.000000,1437.000000,1432.000000,...,21742,21742,0.0,0.0,7719,7719.000000,11358,11358,11358,11358
unique,NaN,NaN,10736,146,NaN,NaN,NaN,NaN,NaN,NaN,...,10,2,NaN,NaN,2,NaN,2,2,2,2
top,NaN,NaN,Piso en venta en El Viso,"Chamartín, Madrid",NaN,NaN,NaN,NaN,NaN,NaN,...,en trámite,False,NaN,NaN,True,NaN,False,False,True,False
freq,NaN,NaN,193,851,NaN,NaN,NaN,NaN,NaN,NaN,...,10937,14023,NaN,NaN,7115,NaN,8567,7287,5823,6302
mean,10870.500000,10871.500000,NaN,NaN,146.920892,103.458192,3.005749,2.091687,3.128740,241.692737,...,NaN,NaN,NaN,NaN,NaN,2658.000518,NaN,NaN,NaN,NaN
std,6276.519112,6276.519112,NaN,NaN,134.181865,88.259192,1.510497,1.406992,0.907713,247.484853,...,NaN,NaN,NaN,NaN,NaN,13360.966258,NaN,NaN,NaN,NaN
min,0.000000,1.000000,NaN,NaN,13.000000,1.000000,0.000000,1.000000,1.000000,1.000000,...,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN
25%,5435.250000,5436.250000,NaN,NaN,70.000000,59.000000,2.000000,1.000000,2.000000,2.000000,...,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN
50%,10870.500000,10871.500000,NaN,NaN,100.000000,79.000000,3.000000,2.000000,3.000000,232.000000,...,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN
75%,16305.750000,16306.750000,NaN,NaN,162.000000,113.000000,4.000000,2.000000,4.000000,354.000000,...,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN


# Handle nulls values 
- We will drop the column if there is more than 50% of missing values or the columns are irrelevant with beetween 10% and 50% missing values
- We will drop the observations if they are less than 10% of missing values  
*Note that every null value must be consider as "Unknown" and not False for boolean columns*

In [3]:
dataframe_observations = dataframe.shape[0]

for column in dataframe.columns:
    if dataframe[column].isnull().sum() >  dataframe_observations * 0.5:
        dataframe.drop(columns=[column], inplace=True)
        print(f"Drop column : {column}")

Drop column : sq_mt_useful
Drop column : n_floors
Drop column : sq_mt_allotment
Drop column : latitude
Drop column : longitude
Drop column : street_number
Drop column : portal
Drop column : door
Drop column : rent_price_by_area
Drop column : built_year
Drop column : are_pets_allowed
Drop column : has_garden
Drop column : has_pool
Drop column : has_terrace
Drop column : has_balcony
Drop column : has_storage_room
Drop column : is_furnished
Drop column : is_kitchen_equipped
Drop column : is_accessible
Drop column : has_green_zones
Drop column : has_private_parking
Drop column : has_public_parking
Drop column : is_parking_included_in_price
Drop column : parking_price


In [4]:
dataframe.isnull().sum().sort_values(ascending=False)

has_ac                     10531
is_orientation_east        10384
is_orientation_south       10384
is_orientation_west        10384
is_orientation_north       10384
has_fitted_wardrobes        8343
has_central_heating         8134
has_individual_heating      8134
street_name                 5905
raw_address                 5465
is_exterior                 3043
floor                       2607
has_lift                    2386
is_floor_under              1170
is_new_development           992
house_type_id                391
sq_mt_built                  126
n_bathrooms                   16
energy_certificate             0
has_parking                    0
Unnamed: 0                     0
is_renewal_needed              0
is_buy_price_known             0
id                             0
buy_price                      0
is_rent_price_known            0
rent_price                     0
operation                      0
neighborhood_id                0
is_exact_address_hidden        0
n_rooms   

# Analysis columns with nulls values left
As we can see, there is still **18 columns** with null values.
- **is_oriented_** columns are not relevant in a regression models, with arround 35% of nulls values : **drop**
- **has_** columns could be relevant but with arround 20% of nulls values, and they could not really be completed by median (but by KNeighbor), it will be too much effort for poor resutls : **drop**
- **stree_name**, **raw_address**, etc... and all string columns : they are only relevant for specific analysis and not for global analysis or regression model : **drop**
- For the columns left, with only arround 5% maximum of nulls values, we can drop the observations
NB : **n_bathrooms** and **sq_mt_built** have less than 10 missing values, so we replace it by the median too minimize performance impact on the model 

In [5]:
DROP_COLUMNS = ["is_orientation_east", "is_orientation_west", "is_orientation_north", "is_orientation_south", "has_fitted_wardrobes", "has_central_heating", "has_individual_heating", "has_lift", "has_ac", "street_name", "raw_address"]

dataframe.drop(columns=DROP_COLUMNS, inplace=True)
dataframe.dropna(subset=["is_exterior", "floor", "house_type_id", "is_new_development"], inplace=True)
dataframe["n_bathrooms"] = dataframe["n_bathrooms"].fillna(value=dataframe["n_bathrooms"].median())
dataframe["sq_mt_built"] = dataframe["sq_mt_built"].fillna(value=dataframe["sq_mt_built"].median())

dataframe.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17536 entries, 0 to 21739
Data columns (total 23 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Unnamed: 0               17536 non-null  int64  
 1   id                       17536 non-null  int64  
 2   title                    17536 non-null  object 
 3   subtitle                 17536 non-null  object 
 4   sq_mt_built              17536 non-null  float64
 5   n_rooms                  17536 non-null  int64  
 6   n_bathrooms              17536 non-null  float64
 7   is_exact_address_hidden  17536 non-null  bool   
 8   floor                    17536 non-null  object 
 9   is_floor_under           17536 non-null  object 
 10  neighborhood_id          17536 non-null  object 
 11  operation                17536 non-null  object 
 12  rent_price               17536 non-null  int64  
 13  is_rent_price_known      17536 non-null  bool   
 14  buy_price             

# Unusable columns 
Some columns are only there to the detail of specific observations but could not be used as global analyse or prediction model so we can drop them.  

In [6]:
print(f"The length of all the value of neighborhood_id is : {len(dataframe['neighborhood_id'].unique())}")
dataframe.drop(columns=['title', 'subtitle', 'id', 'Unnamed: 0', 'is_exact_address_hidden', 'neighborhood_id'], inplace=True)
dataframe.head(1)

The length of all the value of neighborhood_id is : 126


,sq_mt_built,n_rooms,n_bathrooms,floor,is_floor_under,operation,rent_price,is_rent_price_known,buy_price,buy_price_by_area,is_buy_price_known,house_type_id,is_renewal_needed,is_new_development,is_exterior,energy_certificate,has_parking
0,64.0,2,1.0,3,False,sale,471,False,85000,1328,True,HouseType 1: Pisos,False,False,True,D,False


# Outliers & duplicates
To avoid some performances erros, we must check that there are no outliers or duplicates in the dataframe and drop them

In [7]:
dataframe.drop_duplicates(inplace=True)

In [8]:
NUMERICAL_COLUMNS = [column for column in dataframe.columns if dataframe[column].dtype in ["int64", "float64"]]
CATEGORICAL_COLUMNS = [column for column in dataframe.columns if dataframe[column].dtype in ["object", 'bool']]

dataframe = drop_outliers(dataframe=dataframe, columns=NUMERICAL_COLUMNS, percent=90)
dataframe.describe()

,sq_mt_built,n_rooms,n_bathrooms,rent_price,buy_price,buy_price_by_area
count,15285.000000,15285.000000,15285.000000,15285.000000,1.528500e+04,15285.000000
mean,106.996402,2.721753,1.690612,1320.783644,4.195959e+05,3801.811907
std,55.609111,1.087136,0.798778,615.949246,2.996587e+05,1638.622487
min,16.000000,0.000000,1.000000,-1980.000000,3.600000e+04,447.000000
25%,69.000000,2.000000,1.000000,851.000000,1.850000e+05,2491.000000
50%,92.000000,3.000000,2.000000,1204.000000,3.200000e+05,3626.000000
75%,131.000000,3.000000,2.000000,1770.000000,5.750000e+05,4760.000000
max,485.000000,8.000000,6.000000,2517.000000,1.485000e+06,11923.000000


In [9]:
counter_columns(dataframe, CATEGORICAL_COLUMNS)

The counter of floor is : Counter({'1': 3567, '2': 2809, '3': 2377, '4': 1853, 'Bajo': 1740, '5': 1033, '6': 702, '7': 441, '8': 266, 'Entreplanta exterior': 213, '9': 149, 'Semi-sótano exterior': 50, 'Entreplanta interior': 32, 'Semi-sótano interior': 28, 'Sótano interior': 21, 'Sótano exterior': 4})
The counter of is_floor_under is : Counter({False: 13197, True: 2088})
The counter of operation is : Counter({'sale': 15285})
The counter of is_rent_price_known is : Counter({False: 15285})
The counter of is_buy_price_known is : Counter({True: 15285})
The counter of house_type_id is : Counter({'HouseType 1: Pisos': 14000, 'HouseType 5: Áticos': 753, 'HouseType 4: Dúplex': 532})
The counter of is_renewal_needed is : Counter({False: 12344, True: 2941})
The counter of is_new_development is : Counter({False: 14319, True: 966})
The counter of is_exterior is : Counter({True: 13753, False: 1532})
The counter of energy_certificate is : Counter({'en trámite': 7857, 'no indicado': 2439, 'E': 1990, 

### Categorical columns outliers and unique values
As we can see, some columns have only one unique value and are not relevant in any analyse.  
Also, for the column floor to reduce the outliers values, we will map the sotano, the entreplanta and the semi sotano together.

In [10]:
dataframe.drop(columns=["operation", "is_rent_price_known", "is_buy_price_known"], inplace=True)
dataframe['floor'] = dataframe['floor'].apply(unite_floors_together)

# Dump the dataframe to a new cleaned CSV 

In [11]:
dataframe.to_csv('../data/clean/houses_madrid_cleaned.csv', index=False)